# Serverless Agents on Cloud Run Workshop

Welcome! In this workshop, we will build a "Micro-Agent" system using Google Cloud Run, Vertex AI (Gemini 2.5 Flash), and Eventarc.

**Architecture:**
1. **Librarian Service**: Ingests PDFs from Cloud Storage, summarizes them using Gemini, and saves to Firestore.
2. **Guide Service**: Answers user questions using the knowledge in Firestore.

Let's get started!

## 1. Setup & Authentication

**Why do we need this?**
To interact with Google Cloud resources (like Cloud Run, Firestore, etc.) from this notebook, we need to prove who we are. We use `gcloud auth login` to authenticate your personal Google account and set up "Application Default Credentials" (ADC). This allows the Python libraries we use later to automatically find your credentials.

If you don't have a project yet:

1. [Create a project](https://console.cloud.google.com/projectcreate) in the Google Cloud Console.
2. Copy your `Project ID` from the project's [Settings page](https://console.cloud.google.com/iam-admin/settings).

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type:"string", isTemplate: true}
REGION = "us-central1"  # @param {type:"string", isTemplate: true}

if PROJECT_ID == "[your-project-id]" or not PROJECT_ID:
    print("Please specify your project id in PROJECT_ID variable.")
    raise KeyboardInterrupt

!gcloud auth print-identity-token -q &> /dev/null || gcloud auth login --project="{PROJECT_ID}" --update-adc --quiet

!gcloud config set project {PROJECT_ID}
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_REGION"] = REGION

## 1.1 Clone Repository

**Why are we doing this?**
Google Colab is a temporary virtual machine. It starts empty. The code for our agents (`main.py`, `Dockerfile`) lives in GitHub. We need to `git clone` (download) that code into this machine so we can build and deploy it.

In [ ]:
!git clone https://github.com/sangalo20/Severless-agents-cloudrun.git
%cd Severless-agents-cloudrun

## 2. Enable APIs

**What are these?**
Google Cloud services are not enabled by default. We need to turn on the specific services we plan to use:
*   `run.googleapis.com`: **Cloud Run** (to run our containers).
*   `eventarc.googleapis.com`: **Eventarc** (to trigger the Librarian when a file is uploaded).
*   `aiplatform.googleapis.com`: **Vertex AI** (to use the Gemini model).
*   `firestore.googleapis.com`: **Firestore** (our database).
*   `cloudbuild.googleapis.com`: **Cloud Build** (to build our Docker containers).
*   `storage.googleapis.com`: **Cloud Storage** (to store the PDF files).
*   `artifactregistry.googleapis.com`: **Artifact Registry** (to store our Docker images).

In [ ]:
!gcloud services enable run.googleapis.com eventarc.googleapis.com aiplatform.googleapis.com firestore.googleapis.com cloudbuild.googleapis.com storage.googleapis.com artifactregistry.googleapis.com

## 3. Create Infrastructure

**The Plan:**
1.  **Cloud Storage Bucket**: We need a place to upload our conference schedules (PDFs). This bucket will act as the "Inbox" for our Librarian agent.
2.  **Firestore Database**: We need a fast, serverless database to store the *summarized knowledge* and the *chat history*. We use Firestore in "Native" mode.
3.  **Artifact Registry**: We need a repository to store our Docker images.

In [ ]:
BUCKET_NAME = f"{PROJECT_ID}-knowledge-base"
!gsutil mb -l {REGION} gs://{BUCKET_NAME}
print(f"Created bucket: {BUCKET_NAME}")

# Create Firestore in Native mode (if not exists)
!gcloud firestore databases create --location={REGION} --type=firestore-native

# Create Artifact Registry Repository
!gcloud artifacts repositories create containers --repository-format=docker --location={REGION} --description="Docker repository"

## 4. Build "The Librarian" Service

**Step 1: Build Container**
We use `gcloud builds submit` to package our python code (`librarian/main.py`) into a Docker container image. This image is stored in Artifact Registry and is ready to be deployed.

In [ ]:
SERVICE_NAME_LIBRARIAN = "librarian"
!gcloud builds submit --tag {REGION}-docker.pkg.dev/{PROJECT_ID}/containers/{SERVICE_NAME_LIBRARIAN} librarian/

## 4.1 Deploy "The Librarian" Service

**Step 2: Deploy to Cloud Run**
Now we take the image we just built and deploy it to Cloud Run. We use `--allow-unauthenticated` so that Eventarc can easily trigger it.

In [ ]:
!gcloud run deploy {SERVICE_NAME_LIBRARIAN} --image {REGION}-docker.pkg.dev/{PROJECT_ID}/containers/{SERVICE_NAME_LIBRARIAN} --region {REGION} --allow-unauthenticated

## 5. Build "The Guide" Service

**Step 1: Build Container**
Similar to the Librarian, we first build the container image for the Guide service.

In [ ]:
SERVICE_NAME_GUIDE = "guide"
!gcloud builds submit --tag {REGION}-docker.pkg.dev/{PROJECT_ID}/containers/{SERVICE_NAME_GUIDE} guide/

## 5.1 Deploy "The Guide" Service

**Step 2: Deploy to Cloud Run**
We deploy the Guide service. This service will host the chat endpoint.

In [ ]:
!gcloud run deploy {SERVICE_NAME_GUIDE} --image {REGION}-docker.pkg.dev/{PROJECT_ID}/containers/{SERVICE_NAME_GUIDE} --region {REGION} --allow-unauthenticated

## 6. Wire it up with Eventarc

**The Magic Glue**
Right now, the Librarian service is running, but it doesn't know when a file is uploaded. We need **Eventarc** to bridge the gap.

We create a **Trigger** that says:
*   **IF** a file is `finalized` (uploaded) ...
*   **IN** the specific bucket `{BUCKET_NAME}` ...
*   **THEN** send a POST request to the `{SERVICE_NAME_LIBRARIAN}` service.

*Note: We also grant the necessary IAM permissions so Eventarc is allowed to call our Cloud Run service.*

In [ ]:
# Grant permission to the Compute Engine service account (default for Eventarc)
PROJECT_NUMBER = !gcloud projects describe {PROJECT_ID} --format='value(projectNumber)'
PROJECT_NUMBER = PROJECT_NUMBER[0]
SERVICE_ACCOUNT = f"{PROJECT_NUMBER}-compute@developer.gserviceaccount.com"

!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/eventarc.eventReceiver
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role=roles/run.invoker

# Create the trigger
!gcloud eventarc triggers create librarian-trigger \
  --location={REGION} \
  --destination-run-service={SERVICE_NAME_LIBRARIAN} \
  --destination-run-region={REGION} \
  --event-filters="type=google.cloud.storage.object.v1.finalized" \
  --event-filters="bucket={BUCKET_NAME}" \
  --service-account={SERVICE_ACCOUNT}

## 7. Test it!

**Let's see it in action**
1.  **Ingestion**: We create a dummy text file (`schedule.txt`) and upload it to the bucket. This should trigger the Librarian to read it, summarize it with Gemini, and save it to Firestore.
2.  **Chat**: We send a chat message to the Guide service. It should look up the summary in Firestore and answer our question.

In [ ]:
# 1. Upload a dummy schedule
with open("schedule.txt", "w") as f:
    f.write("DevFest Schedule:\n10:00 AM - Keynote by Google\n11:00 AM - Serverless Agents Workshop\n12:00 PM - Lunch")

!gsutil cp schedule.txt gs://{BUCKET_NAME}/schedule.txt
print("File uploaded. Waiting for Eventarc (approx 1-2 mins)...")

In [ ]:
# 2. Chat with the Guide
import requests
import time

# Get Guide URL
GUIDE_URL = !gcloud run services describe {SERVICE_NAME_GUIDE} --region {REGION} --format='value(status.url)'
GUIDE_URL = GUIDE_URL[0]

print(f"Chatting with Guide at: {GUIDE_URL}")

query = {"session_id": "test-session", "query": "What time is the Serverless Agents workshop?"}
response = requests.post(f"{GUIDE_URL}/chat", json=query)
print(response.json())